In [10]:
import cv2
import numpy as np
import pytesseract
image = cv2.imread('../raw_data/label_composition_images/IMG_1388.JPG')

In [11]:
def preprocessing_image(image):
    #image = cv2.imread('../raw_data/label_composition_images/IMG_1378.JPG')
    grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    noise = cv2.medianBlur(grayscale,5)
    #threshold = cv2.threshold(noise, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    #threshold doesn't really work

    #dilation
    kernel = np.ones((5,5),np.uint8)
    dilate = cv2.dilate(noise, kernel, iterations = 1)

    #erosion

    kernel = np.ones((5,5),np.uint8)
    erode= cv2.erode(dilate, kernel, iterations = 1)

    #opening - erosion followed by dilation
    #definition opening(image):
    #kernel = np.ones((5,5),np.uint8)
    #return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

    #canny edge detection

    #canny = cv2.Canny(threshold, 100, 200)

    #skew correction
    #definition deskew(image):
    coords = np.column_stack(np.where(erode > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = erode.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    deskew = cv2.warpAffine(erode, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)


    #template matching
    #definition match_template(image, template):
    #return cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)

    #definition sharp(image):
    #sharp= cv2.bilateralFilter(deskew,9,75,75)
    return deskew

In [12]:
def image_to_string(image):
    
    preprocessed = preprocessing_image(image)
   
        #img = Image.fromarray(preprocessed, 'RGB')
    results = pytesseract.image_to_string(preprocessed)
    return results

In [13]:
image_to_string(image)

"Aw'1g 05/19\n\n36% Lana / Wool / Wolle / Laine /\nLana/ Ls\n\n35% Acrilica f Acrylic | Polyacrl /\nActylique / Actilicg / Aerflico\n\n18% Alpaca / Alpaca / Alpaka /\nAlpaga / Alnaca / Alpaca\n\n13% Poliamida / Nylon / Polyamid /\nFolyamide / Poliammidica / Polyamide\n\n1% Elastano / Elastane / Elasthan/\nElasthanne / Elastan / Elastano\n\nWAR ARE\n\nLAVAR AMANO - NO DEJAR OIC\nNO USAR LEJIA BN REM\n\nNO UTILIZAR SECADORA\n\nPLANCHAR A TEMPERATURA BAJA\n\nMAX. 141°C\n\nNO LIMPIAR EN SECO\n\nLAVAR OEL REVES\n\nPLANCHAR DEL REVES\n\nSECAR =N PLANO\n\x0c"